In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
from ipywidgets import IntProgress
from IPython.display import display

In [3]:
from model_builder import ModelBuilder
from solve_action import SolveAction

In [4]:
from dependencies_calculator import DependenciesCalculator
from cost_calculator_factory import CostCalculatorFactory
from pyomo_values_converter import PyomoValuesConverter

In [5]:
M = 10 ** 6

In [6]:
read_matrix = lambda path: pd.read_csv(path, index_col=0, encoding='utf-8').to_numpy()

In [7]:
C = read_matrix('../publication_4/script/out/C.csv')

In [8]:
D = read_matrix('../publication_4/script/out/D.csv')

In [9]:
Q = read_matrix('../publication_4/script/out/Q.csv')

In [10]:
R = read_matrix('../publication_4/script/out/R.csv')

In [12]:
constants_file = open('../publication_4/script/out/constants.json', 'r', encoding='utf-8')
constants_data = constants_file.read()
constants_file.close()
constants = json.loads(constants_data)

In [15]:
n = constants['n']
m = constants['m']
l = constants['l']

In [ ]:
k_to_cost = {}

In [19]:
solver_name = 'glpk'

In [20]:
for k in range(1, l + 1):
    calculate_deps_progress = IntProgress(min=0, max=m, value=0, description='deps', bar_style='success')
    display(calculate_deps_progress)
    deps_calculator = DependenciesCalculator(D, m, calculate_deps_progress)
    sum_D = deps_calculator.calculate()
    model_builder = ModelBuilder(M, k, Q, sum_D, C, R)
    model = model_builder.build()
    solve_action = SolveAction(model, solver_name)
    solve_result = solve_action.solve()
    instance = solve_result.instance
    pyomo_values_converter = PyomoValuesConverter()
    X = pyomo_values_converter.convert(instance.A)
    cost_calculator_factory = CostCalculatorFactory(Q, sum_D, C, R)
    cost_calculator = cost_calculator_factory.get_cost_calculator(X)
    cost = cost_calculator.calculate()
    k_to_cost[k] = cost

IntProgress(value=0, bar_style='success', description='deps', max=124)

solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
k_to_cost